In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os
import glob

In [2]:
results_folder = "results"
df_experiment = pd.DataFrame()

In [3]:
for experiment_folder in os.listdir(results_folder):
    experiment_files = os.listdir(os.path.join(results_folder,experiment_folder))

    if len(experiment_files) != 3:
        continue
    
    for file in experiment_files:
        
        read_file = os.path.join(results_folder,experiment_folder,file)

        
        
        if "config" in file:
            df_config =pd.read_csv(read_file)
            
        if "test_metrics" in file:
            df_test = pd.read_csv(read_file)
            
        if "benchmark" in file:
            df_benchmark = pd.read_csv(read_file)
            
            avg_time = df_benchmark.Time.mean()
            avg_pop = df_benchmark.Avg.iloc[-1]
            best_pop = df_benchmark.Best.iloc[-1]
            worst_pop = df_benchmark.Worst.iloc[-1]
            
            df_benchmark = pd.DataFrame({"Time": [avg_time],"Avg":[avg_pop],"Worst":[worst_pop],"Best":[best_pop]})

    row = pd.concat((df_test,df_benchmark,df_config),axis=1)

    df_experiment = pd.concat((df_experiment,row))

df_experiment = df_experiment.reset_index(drop=True)

In [4]:
df_experiment.columns

Index(['F1', 'Accuracy', 'ROC AUC', 'Best Chromosome', 'Time', 'Avg', 'Worst',
       'Best', 'dataset', 'crossover_choice', 'mutation_rate', 'metric_choice',
       'population_size', 'elitism', 'evolution_rounds', 'stopping_threshold',
       'model', 'algorithm', 'backend_prefer'],
      dtype='object')

In [5]:
models = ["mlp","xgboost","logistic"]
datasets = ["gina_agnostic","hiva_agnostic","sylva_agnostic"]
algorithms = ["baseline_metrics","rfs","ga_seq","random","ga_joblib","ga_spark"][::-1]

In [6]:
df_experiment.groupby(["dataset","model"])[["F1","Accuracy","ROC AUC"]].mean().reindex(index=models, level=1)

F1  Accuracy   ROC AUC
dataset        model                                 
gina_agnostic  mlp       0.892759  0.895677  0.895465
               xgboost   0.934043  0.935447  0.935334
               logistic  0.797616  0.801633  0.801516
hiva_agnostic  mlp       0.525535  0.974783  0.697958
               xgboost   0.390510  0.969661  0.636438
               logistic  0.484918  0.968873  0.700245
sylva_agnostic mlp       0.960810  0.995139  0.981666
               xgboost   0.951203  0.993981  0.974052
               logistic  0.941223  0.992857  0.963207

In [16]:
df_metrics = df_experiment[df_experiment.backend_prefer=="processes"].reset_index(drop=True).groupby(["dataset","model","algorithm"])[["F1","Accuracy","ROC AUC"]].mean().unstack(level="dataset").reindex(index=algorithms, level=1).reindex(index=models,level=0).stack(0).unstack()#.swaplevel(0, 1).sort_index(axis=1, level=0)
df_metrics = df_metrics.applymap(lambda x: str(np.round(x,3)))
df_metrics

dataset                   gina_agnostic                hiva_agnostic         \
                               Accuracy     F1 ROC AUC      Accuracy     F1   
model    algorithm                                                            
mlp      ga_spark                 0.934  0.931   0.933         0.983  0.696   
         ga_joblib                0.893  0.891   0.893         0.972    0.5   
         random                   0.879  0.876   0.879           nan    nan   
         ga_seq                   0.893  0.891   0.893           nan    nan   
         baseline_metrics         0.879  0.875   0.879         0.969  0.381   
xgboost  ga_spark                 0.971  0.971   0.971         0.981  0.636   
         ga_joblib                0.925  0.924   0.925         0.967  0.364   
         random                     nan    nan     nan         0.967    0.3   
         ga_seq                   0.925  0.924   0.925         0.967  0.364   
         baseline_metrics         0.931  0.928    0.93         0.969  0.316   
logistic ga_spark                   nan    nan     nan         0.988    0.8   
         ga_joblib                0.804  0.795   0.804         0.962  0.429   
         random                   0.795  0.793   0.795         0.962  0.385   
         ga_seq                   0.804  0.795   0.804         0.967  0.462   
         rfs                      0.801  0.801   0.801           nan    nan   
         baseline_metrics         0.798  0.797   0.798         0.965    0.4   

dataset                           sylva_agnostic                 
                          ROC AUC       Accuracy     F1 ROC AUC  
model    algorithm                                               
mlp      ga_spark           0.767          0.999  0.989   0.999  
         ga_joblib          0.696          0.994   0.95   0.976  
         random               nan            nan    nan     nan  
         ga_seq               nan            nan    nan     nan  
         baseline_metrics   0.631          0.993  0.944    0.97  
xgboost  ga_spark           0.733          0.997  0.972   0.993  
         ga_joblib           0.63          0.994  0.955   0.976  
         random             0.598          0.992  0.938   0.964  
         ga_seq              0.63          0.994  0.955   0.976  
         baseline_metrics   0.599          0.992  0.932   0.959  
logistic ga_spark           0.833          0.999  0.994   0.994  
         ga_joblib          0.691          0.992  0.931   0.954  
         random             0.659          0.994  0.949    0.97  
         ga_seq             0.694          0.993  0.944    0.97  
         rfs                  nan          0.988  0.895    0.93  
         baseline_metrics   0.661          0.993  0.944    0.97

In [8]:
df_metrics.rename(level="algorithm",mapper={algo:algo.replace("_","\_") for algo in algorithms},inplace=True)
df_metrics.rename(level="algorithm",mapper={"baseline\_metrics":"baseline"},inplace=True)
df_metrics.rename(level="dataset",columns={data:data.replace("_","\_") for data in datasets},inplace=True)
print(df_metrics.to_latex(index=True,multicolumn_format="c"))

\begin{tabular}{lllllllllll}
\toprule
 & dataset & \multicolumn{3}{c}{gina\_agnostic} & \multicolumn{3}{c}{hiva\_agnostic} & \multicolumn{3}{c}{sylva\_agnostic} \\
 &  & Accuracy & F1 & ROC AUC & Accuracy & F1 & ROC AUC & Accuracy & F1 & ROC AUC \\
model & algorithm &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{5}{*}{mlp} & ga\_spark & 0.934 & 0.931 & 0.933 & 0.983 & 0.696 & 0.767 & 0.999 & 0.989 & 0.999 \\
 & ga\_joblib & 0.893 & 0.891 & 0.893 & 0.972 & 0.5 & 0.696 & 0.994 & 0.95 & 0.976 \\
 & random & 0.879 & 0.876 & 0.879 & nan & nan & nan & nan & nan & nan \\
 & ga\_seq & 0.893 & 0.891 & 0.893 & nan & nan & nan & nan & nan & nan \\
 & baseline & 0.879 & 0.875 & 0.879 & 0.969 & 0.381 & 0.631 & 0.993 & 0.944 & 0.97 \\
\cline{1-11}
\multirow[t]{5}{*}{xgboost} & ga\_spark & 0.971 & 0.971 & 0.971 & 0.981 & 0.636 & 0.733 & 0.997 & 0.972 & 0.993 \\
 & ga\_joblib & 0.925 & 0.924 & 0.925 & 0.967 & 0.364 & 0.63 & 0.994 & 0.955 & 0.976 \\
 & random & nan & nan & nan & 0.967 & 0.3 & 0.59

In [10]:
df_times = df_experiment[df_experiment.backend_prefer=="processes"].reset_index(drop=True).groupby(["dataset","model","algorithm"])[["Time"]].mean().unstack(level="dataset").reindex(index=algorithms,level=1).reindex(index=models,level=0)
df_times = df_times.applymap(lambda x: str(np.round(x,3)))
df_times.columns = df_times.columns.droplevel(0)
df_times

dataset                   gina_agnostic hiva_agnostic sylva_agnostic
model    algorithm                                                  
mlp      ga_spark                44.906        34.848         20.625
         ga_joblib               26.633         35.05         20.806
         random                 275.861           nan            nan
         ga_seq                 252.604           nan            nan
         baseline_metrics         2.259         5.301          3.589
xgboost  ga_spark                23.849        31.482         24.194
         ga_joblib                22.16        34.257         25.515
         random                     nan        72.523        227.809
         ga_seq                  90.887        72.568         66.404
         baseline_metrics         0.872         1.177          0.869
logistic ga_spark                   nan          5.25          2.593
         ga_joblib                9.455         9.166          2.775
         random                  12.028        11.918         19.987
         ga_seq                   7.429        11.679         22.798
         rfs                  44954.382           nan      16359.015
         baseline_metrics         0.117         0.186           0.37

## Jaccard Overlap

In [11]:
df_times.rename(level="algorithm",mapper={algo:algo.replace("_","\_") for algo in algorithms},inplace=True)
df_times.rename(level="algorithm",mapper={"baseline\_metrics":"baseline"},inplace=True)
df_times.rename(level="dataset",columns={data:data.replace("_","\_") for data in datasets},inplace=True)
print(df_times.to_latex(index=True,multicolumn_format="c",multirow=True))

\begin{tabular}{lllll}
\toprule
 & dataset & gina\_agnostic & hiva\_agnostic & sylva\_agnostic \\
model & algorithm &  &  &  \\
\midrule
\multirow[t]{5}{*}{mlp} & ga\_spark & 44.906 & 34.848 & 20.625 \\
 & ga\_joblib & 26.633 & 35.05 & 20.806 \\
 & random & 275.861 & nan & nan \\
 & ga\_seq & 252.604 & nan & nan \\
 & baseline & 2.259 & 5.301 & 3.589 \\
\cline{1-5}
\multirow[t]{5}{*}{xgboost} & ga\_spark & 23.849 & 31.482 & 24.194 \\
 & ga\_joblib & 22.16 & 34.257 & 25.515 \\
 & random & nan & 72.523 & 227.809 \\
 & ga\_seq & 90.887 & 72.568 & 66.404 \\
 & baseline & 0.872 & 1.177 & 0.869 \\
\cline{1-5}
\multirow[t]{6}{*}{logistic} & ga\_spark & nan & 5.25 & 2.593 \\
 & ga\_joblib & 9.455 & 9.166 & 2.775 \\
 & random & 12.028 & 11.918 & 19.987 \\
 & ga\_seq & 7.429 & 11.679 & 22.798 \\
 & rfs & 44954.382 & nan & 16359.015 \\
 & baseline & 0.117 & 0.186 & 0.37 \\
\cline{1-5}
\bottomrule
\end{tabular}



In [12]:
import itertools

In [13]:
list(itertools.product(algorithms,algorithms))

[('ga_spark', 'ga_spark'),
 ('ga_spark', 'ga_joblib'),
 ('ga_spark', 'random'),
 ('ga_spark', 'ga_seq'),
 ('ga_spark', 'rfs'),
 ('ga_spark', 'baseline_metrics'),
 ('ga_joblib', 'ga_spark'),
 ('ga_joblib', 'ga_joblib'),
 ('ga_joblib', 'random'),
 ('ga_joblib', 'ga_seq'),
 ('ga_joblib', 'rfs'),
 ('ga_joblib', 'baseline_metrics'),
 ('random', 'ga_spark'),
 ('random', 'ga_joblib'),
 ('random', 'random'),
 ('random', 'ga_seq'),
 ('random', 'rfs'),
 ('random', 'baseline_metrics'),
 ('ga_seq', 'ga_spark'),
 ('ga_seq', 'ga_joblib'),
 ('ga_seq', 'random'),
 ('ga_seq', 'ga_seq'),
 ('ga_seq', 'rfs'),
 ('ga_seq', 'baseline_metrics'),
 ('rfs', 'ga_spark'),
 ('rfs', 'ga_joblib'),
 ('rfs', 'random'),
 ('rfs', 'ga_seq'),
 ('rfs', 'rfs'),
 ('rfs', 'baseline_metrics'),
 ('baseline_metrics', 'ga_spark'),
 ('baseline_metrics', 'ga_joblib'),
 ('baseline_metrics', 'random'),
 ('baseline_metrics', 'ga_seq'),
 ('baseline_metrics', 'rfs'),
 ('baseline_metrics', 'baseline_metrics')]

In [ ]:
from scipy.spatial.distance import jaccard

In [ ]:
def feature_jaccard_overlap(df,dataset,algorithms):
    pairwise_comparisons = list(itertools.product(algorithms,algorithms))
    df = df[df.dataset==dataset]
    for algo1,algo2 in pairwise_comparisons:
        try:
            chromosome1 = eval(df[df.algorithm==algo1]["Best Chromosome"].values[0].replace(" ",","))
            chromosome2 = eval(df[df.algorithm==algo2]["Best Chromosome"].values[0].replace(" ",","))
            # print(chromosome1,chromosome2)
            print(algo1,algo2,1-jaccard(chromosome1,chromosome2))
        except:
            continue

df_subset = df_experiment[(df_experiment.backend_prefer=="processes") & (df_experiment.model=="logistic")].reset_index(drop=True)
feature_jaccard_overlap(df=df_subset,dataset="sylva_agnostic",algorithms=algorithms)

## EXAMPLE IMAGE

In [ ]:
from colorama import Fore, Back, Style
from IPython.display import display, Latex

In [ ]:
from sklearn.datasets import load_iris
X,y = load_iris(return_X_y=True,as_frame=True)
print(Fore.RED + "Full Dataset")
print(Fore.BLACK + f"{X[:5].to_string(index=False)}")
print()

c_l = np.array([0,0,1,1])
print(Fore.RED + "\nChromosome l")
display(Latex('$c_{l}$='+f'{c_l}'))
print()

X_ga = X.iloc[:,c_l==1]

print(Fore.RED + "Genetic Algorithm Subset")
print(Fore.BLACK + f"{X_ga[:5].to_string(index=False)}")